# Data preparation, no need to run

In [ ]:
### Setup the Environment

# !pip install numpy pandas tensorflow keras opencv-python scikit-learn matplotlib
# !pip install torch torchvision


In [4]:
import os
import pandas as pd
import numpy as np
import cv2
from tensorflow.keras.utils import to_categorical

# Paths
train_annotations_path = './train_coco_annotations.csv'
train_folder = './data/train/'
valid_annotations_path = './valid_coco_annotations.csv'
valid_folder = './data/valid/'

# Function to load data
def load_data(annotations_path, folder):
    annotations = pd.read_csv(annotations_path)
    image_paths = []
    labels = []

    for _, row in annotations.iterrows():
        image_id = row['FileName']
        category = row['Category']  # 0 for holds, 1 for volumes
        image_path = os.path.join(folder, f"{image_id}")

        if os.path.exists(image_path):
            image_paths.append(image_path)
            labels.append(category)
        else:
            print(f"Image not found: {image_path}")

    return image_paths, labels

# Load training data
train_image_paths, train_labels = load_data(train_annotations_path, train_folder)

# Load validation data
valid_image_paths, valid_labels = load_data(valid_annotations_path, valid_folder)

# Convert labels to numeric
unique_classes = sorted(set(train_labels + valid_labels))
class_to_idx = {cls: idx for idx, cls in enumerate(unique_classes)}
train_labels_numeric = [class_to_idx[label] for label in train_labels]
valid_labels_numeric = [class_to_idx[label] for label in valid_labels]

# Print summary
print(f"Number of training samples: {len(train_image_paths)}")
print(f"Number of validation samples: {len(valid_image_paths)}")


Number of training samples: 129868
Number of validation samples: 11335


In [ ]:
# # debugging codes
# print(f"Labels: {labels}")
# print(f"Unique Classes: {unique_classes}")
# print(f"Class to Index Mapping: {class_to_idx}")
# print(f"Labels Numeric: {labels_numeric}")

# for path in image_paths:
#     if not os.path.exists(path):
#         print(f"Missing file: {path}")

# print(f"Train Folder: {train_folder}")
# print(os.listdir(train_folder))

# print(f"Annotations Dataset Shape: {annotations.shape}")
# print(annotations.head())
# print(f"Columns in Dataset: {annotations.columns}")
# print(annotations['Category'].unique())  # Replace 'label_column' with the actual column name

# labels = annotations['Category'].tolist()  # Replace 'label_column' with the actual column name
# print(f"Extracted Labels: {labels}")



In [16]:
import os
import shutil
import pandas as pd
import cv2

# Paths
train_annotations_path = './train_coco_annotations.csv'
valid_annotations_path = './valid_coco_annotations.csv'
train_folder = './data/train/'
valid_folder = './data/valid/'
yolo_data_folder = './yolo_data/'

# Create new YOLO data folders
train_images_folder = os.path.join(yolo_data_folder, 'train/images/')
train_labels_folder = os.path.join(yolo_data_folder, 'train/labels/')
valid_images_folder = os.path.join(yolo_data_folder, 'valid/images/')
valid_labels_folder = os.path.join(yolo_data_folder, 'valid/labels/')

os.makedirs(train_images_folder, exist_ok=True)
os.makedirs(train_labels_folder, exist_ok=True)
os.makedirs(valid_images_folder, exist_ok=True)
os.makedirs(valid_labels_folder, exist_ok=True)

# Function to copy and convert COCO annotations to YOLO format
def process_annotations(annotations_path, src_folder, dest_images_folder, dest_labels_folder, dataset_name):
    annotations = pd.read_csv(annotations_path)
    total_annotations = len(annotations)

    for idx, row in enumerate(annotations.iterrows(), start=1):
        _, row_data = row
        image_id = row_data['FileName']
        bbox = eval(row_data['BBox'])  # Convert BBox string to list
        class_id = row_data['Category']  # 0 for holds, 1 for volumes
        src_image_path = os.path.join(src_folder, image_id)
        dest_image_path = os.path.join(dest_images_folder, image_id)
        yolo_label_path = os.path.join(dest_labels_folder, os.path.splitext(image_id)[0] + ".txt")

        # Copy image
        if os.path.exists(src_image_path):
            shutil.copy(src_image_path, dest_image_path)

            # Load image to get dimensions
            image = cv2.imread(src_image_path)
            if image is not None:
                img_height, img_width = image.shape[:2]

                # Convert bbox to YOLO format
                x, y, w, h = bbox
                x_center = (x + w / 2) / img_width
                y_center = (y + h / 2) / img_height
                width = w / img_width
                height = h / img_height

                # Write YOLO annotation
                with open(yolo_label_path, "a") as yolo_file:
                    yolo_file.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")
            else:
                print(f"Could not load image: {src_image_path}")
        else:
            print(f"Image not found: {src_image_path}")

        # Calculate and display progress
        progress = (idx / total_annotations) * 100
        print(f"[{dataset_name}] Progress: {progress:.2f}% ({idx}/{total_annotations})", end="\r")

    print(f"\n[{dataset_name}] Processing completed.")

# Process training data
process_annotations(train_annotations_path, train_folder, train_images_folder, train_labels_folder, "Training")

# Process validation data
process_annotations(valid_annotations_path, valid_folder, valid_images_folder, valid_labels_folder, "Validation")

print("Dataset organized and annotations converted to YOLO format.")


KeyboardInterrupt: 

# Run YOLO in yolov5 folder

- Code for running YOLO  
python train.py --img 640 --batch 16 --epochs 50 --data data.yaml --weights yolov5s.pt --project runs --name hold_volume_detection

- current:   
python train.py --img 416 --batch 8 --epochs 10 --data data.yaml --weights yolov5n.pt --project runs --name hold_volume_detection --workers 4
Results saved to runs\hold_volume_detection5

- to resume a run  
python train.py --resume

- can try this  
python train.py --img 416 --batch 8 --epochs 10 --data data.yaml --weights yolov5n.pt --hyp hyp.scratch-low.yaml --workers 4



- Results saved to runs\hold_volume_detection5

10 epochs completed in 0.818 hours.
Optimizer stripped from runs\hold_volume_detection5\weights\last.pt, 3.7MB
Optimizer stripped from runs\hold_volume_detection5\weights\best.pt, 3.7MB

Validating runs\hold_volume_detection5\weights\best.pt...
Fusing layers... 
Model summary: 157 layers, 1761871 parameters, 0 gradients, 4.1 GFLOPs
                 Class     Images  Instances          P          R      mAP50   mAP50-95: 100%|██████████| 18/18 [00:09<00:00,  1.93it/s]
                   all        277      11335      0.782      0.616      0.683      0.409
                  hold        277      10421      0.755      0.638      0.665       0.38
                volume        277        914      0.809      0.595        0.7      0.439
Results saved to runs\hold_volume_detection5

# Make predictions

In [ ]:
import sys
import os
import cv2
import torch
from tqdm import tqdm
import pandas as pd

# Determine the current working directory
script_dir = os.getcwd()


# Change directory to the YOLOv5 folder
yolo_folder = os.path.join(script_dir, "yolov5")
os.chdir(yolo_folder)

In [33]:


# Verify the working directory
print(f"Changed working directory to: {os.getcwd()}")

# Add YOLOv5 folder to Python path
yolov5_path = os.path.join(script_dir, "yolov5")
sys.path.append(yolov5_path)

# Import YOLOv5 components
from models.common import DetectMultiBackend

# Set paths
model_path = os.path.join(script_dir, "runs/hold_volume_detection5/weights/best.pt")
input_folder = os.path.join(script_dir, "data/test_image")
output_folder = os.path.join(script_dir, "data/test_output")
os.makedirs(output_folder, exist_ok=True)

print("Setup complete!")

Changed working directory to: d:\zqw\2024fall\SI670\si670_final_project\yolov5
Setup complete!


In [38]:
import numpy as np

# Load the YOLOv5 model
model = DetectMultiBackend(model_path, device=torch.device('cpu'))
print("Model loaded successfully!")

# Initialize statistics
stats = {"class": [], "confidence": [], "image": [], "x1": [], "y1": [], "x2": [], "y2": [], "cls": []}

# Loop through all images in the input folder
image_files = [f for f in os.listdir(input_folder) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
for image_name in tqdm(image_files, desc="Processing images", unit="image"):
    try:
        image_path = os.path.join(input_folder, image_name)

        # Load image
        img = cv2.imread(image_path)
        if img is None:
            print(f"Warning: Unable to read image {image_name}. Skipping.")
            continue
        img_height, img_width = img.shape[:2]

        # Preprocess image for YOLO
        img_resized = cv2.resize(img, (416, 416))  # Resize to YOLO input size
        img_tensor = torch.from_numpy(img_resized).permute(2, 0, 1).float().div(255.0).unsqueeze(0)  # Normalize and add batch dimension

        # Run inference
        results = model(img_tensor)

        # Process predictions
        for det in results[0]:  # Get the first level of predictions
            # Ensure det is a tensor and process it
            if isinstance(det, torch.Tensor):
                det = det.cpu().numpy()  # Convert tensor to numpy array

            for pred in det:  # Iterate through individual predictions
                # Extract relevant values: bbox, confidence, and class
                if pred.shape[0] >= 6:  # Ensure there are enough elements
                    x_center, y_center, width, height = pred[:4]
                    conf = pred[4]  # Fifth value is confidence
                    cls = pred[5]  # Sixth value is class index

                    

                    # Confidence threshold
                    if conf < 0.5:  # Ignore predictions with low confidence
                        print(f"Skipped prediction with low confidence: {conf}")
                        continue

                    # Convert from center-size format to corner coordinates
                    x1 = int((x_center - width / 2) / 416 * img_width)
                    y1 = int((y_center - height / 2) / 416 * img_height)
                    x2 = int((x_center + width / 2) / 416 * img_width)
                    y2 = int((y_center + height / 2) / 416 * img_height)

                    class_id = 0 if cls <= 0.5 else 1  # 0 for holds, 1 for volumes

                    # Save bounding box locations
                    stats["class"].append(int(class_id))
                    stats["confidence"].append(float(conf))
                    stats["image"].append(image_name)
                    stats["x1"].append(x1)
                    stats["y1"].append(y1)
                    stats["x2"].append(x2)
                    stats["y2"].append(y2)
                    stats["cls"].append(cls)

                    label = f"{'Hold' if cls <= 0.5 else 'Volume'} {conf:.2f}"
                    color = (255, 0, 0) if label=='Hold' else (0, 0, 255)  # Blue for holds, red for volumes
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
                    # Draw bounding boxes
                    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Save the annotated image
        output_path = os.path.join(output_folder, image_name)
        cv2.imwrite(output_path, img)

    except Exception as e:
        print(f"Error processing {image_name}: {e}")

# Save statistics to a CSV file
stats_df = pd.DataFrame(stats)
stats_df.to_csv(os.path.join(output_folder, "test_statistics.csv"), index=False)

print(f"Processed {len(image_files)} images. Results saved to {output_folder}.")


ModuleNotFoundError: No module named 'numpy.strings'

In [35]:
import pandas as pd
import os

def iou(box1, box2):
    """
    Calculate Intersection over Union (IoU) between two bounding boxes.
    box1, box2: (x1, y1, x2, y2)
    """
    x1_inter = max(box1[0], box2[0])
    y1_inter = max(box1[1], box2[1])
    x2_inter = min(box1[2], box2[2])
    y2_inter = min(box1[3], box2[3])

    inter_area = max(0, x2_inter - x1_inter + 1) * max(0, y2_inter - y1_inter + 1)
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    union_area = box1_area + box2_area - inter_area

    return inter_area / union_area if union_area != 0 else 0

def non_maximum_suppression(df, iou_threshold=0.5):
    """
    Apply Non-Maximum Suppression (NMS) to the bounding boxes in a DataFrame.
    df: DataFrame containing 'x1', 'y1', 'x2', 'y2', 'confidence', 'class', 'image'
    iou_threshold: IoU threshold to suppress overlapping boxes
    """
    nms_results = []

    # Process each image separately
    for image, group in df.groupby('image'):
        boxes = group[['x1', 'y1', 'x2', 'y2']].values
        confidences = group['confidence'].values
        classes = group['class'].values

        # Sort boxes by confidence score in descending order
        indices = confidences.argsort()[::-1]
        boxes = boxes[indices]
        confidences = confidences[indices]
        classes = classes[indices]
        group = group.iloc[indices]  # Sort the group DataFrame accordingly

        suppressed = set()
        for i in range(len(boxes)):
            if i in suppressed:
                continue
            current_box = boxes[i]
            nms_results.append(group.iloc[i])  # Keep the box with the highest confidence
            for j in range(i + 1, len(boxes)):
                if j in suppressed:
                    continue
                iou_score = iou(current_box, boxes[j])
                # Suppress boxes based on IoU only, ignoring class differences
                if iou_score > iou_threshold:
                    suppressed.add(j)

    # Convert the results back to a DataFrame
    return pd.DataFrame(nms_results)

# Load the statistics DataFrame
stats_path = os.path.join(output_folder, "test_statistics.csv")
stats_df = pd.read_csv(stats_path)

# Apply NMS
nms_df = non_maximum_suppression(stats_df, iou_threshold=0.5)

# Save the NMS results to a new CSV file
nms_path = os.path.join(output_folder, "nms_statistics.csv")
nms_df.to_csv(nms_path, index=False)

print(f"NMS applied. Results saved to {nms_path}.")


NMS applied. Results saved to d:\zqw\2024fall\SI670\si670_final_project\yolov5\data/test_output\nms_statistics.csv.


In [ ]:
# Create output folder for NMS results
nms_output_folder = os.path.join(output_folder, "nms_output")
os.makedirs(nms_output_folder, exist_ok=True)

# Load NMS results
nms_df = pd.read_csv(os.path.join(output_folder, "nms_statistics.csv"))

# Loop through each image and draw bounding boxes
for image_name, group in nms_df.groupby('image'):
    image_path = os.path.join(input_folder, image_name)
    output_path = os.path.join(nms_output_folder, image_name)

    # Load image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Warning: Unable to read image {image_name}. Skipping.")
        continue

    # Draw bounding boxes from NMS
    for _, row in group.iterrows():
        x1, y1, x2, y2 = int(row['x1']), int(row['y1']), int(row['x2']), int(row['y2'])
        cls = float(row['cls'])
        conf = float(row['confidence'])

        # Define color: blue for holds (class 0), red for volumes (class 1)
        color = (255, 0, 0) if cls >= 0.5 else (0, 0, 255)
        label = f"{'Hold' if cls >= 0.5 else 'Volume'} {conf:.2f}"

        # Draw rectangle and label
        cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)
        cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    # Save the annotated image
    cv2.imwrite(output_path, img)

print(f"NMS drawings saved to: {nms_output_folder}")
